# Scraping [Cultura](https://www.cultura.com/boutiques/des-prix-des-promos.html)

## Initialisation

In [ ]:
import time
import random
import re
from pprint import pprint

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from pymongo import MongoClient

In [ ]:
browser = webdriver.Chrome('/usr/local/bin/chromedriver')

url_to_scrap = "https://www.ikea.com/fr/fr/campaigns/les-indispensables-a-petits-prix-pub0d9cd6c0?icid=fr|20210315|menu|indispensables"

browser.get(url_to_scrap)

# Click to accept cookies
time.sleep(2)
accept_cookie_button = browser.find_element_by_css_selector('#onetrust-accept-btn-handler')
accept_cookie_button.click()

## Cliquer sur chaque lien de promotion

In [ ]:
def random_sleep() :
    ''' Pause aléatoire de maximum 3 secondes '''
    random_number = random.random() * 3 # entre 0 et 3 secondes
    time.sleep(random_number)

In [ ]:
def get_products(browser):
    elements = browser.find_elements_by_css_selector('.range-revamp-product-compact')
    data_elements = []
    for elt in elements:
        data_element = {}
        data_element['marque'] =  elt.find_element_by_class_name('range-revamp-header-section__title--small').text
        data_element['price'] = elt.find_element_by_class_name('range-revamp-price').text
        data_element['nom'] = elt.find_element_by_class_name('range-revamp-header-section__description-text').text
        data_element['ratings'] = elt.find_element_by_class_name('range-revamp-average-rating').get_attribute('aria-label')
        data_element['comments'] = elt.find_element_by_class_name('range-revamp-average-rating__reviews').text

        data_element['price'] = re.findall('([\d+,]+)', data_element['price'])[0]
        data_element['comments'] = data_element['comments'].strip('()')
        data_element['ratings'] = (re.findall('Avis:\s([\d+.]+)', data_element['ratings'])[0])
        data_elements.append(data_element)
    return data_elements

In [ ]:
def parse_urls() :
    ''' Clique sur les urls '''
    print("Browsing...")

    selector_css = ".c130l69h .hqs7k6k > a"
    links = browser.find_elements_by_css_selector(selector_css)

    data_list = []

    for index in range(0, len(links)) :
        random_sleep()
        url = browser.find_elements_by_css_selector(selector_css)[index]
        url.click()
        products = get_products(browser)
        data_list.append(products)
        time.sleep(3)
        browser.back()

    print("Browsing done.")

    return data_list

# Execution
IKEA_data = parse_urls()
#pprint(IKEA_data)

In [ ]:
# Connexion MongoDB
try:
    client = MongoClient('localhost', 27017)
    db = client.IKEA_scraping
    products = db.products
except:
    print("Ooups! la connexion n'a pas pu abouti")

In [58]:
for index, data in enumerate(IKEA_data):
    json_to_import = {
        "title": index,
        "data": data 
    }
    
    # Envoi vers MongoDB
    products.insert_one(json_to_import)

    pprint(json_to_import)

{'data': [{'comments': '164',
           'marque': 'BÄSTIS',
           'nom': 'Brosse adhésive',
           'price': '1',
           'ratings': '4.6'},
          {'comments': '60',
           'marque': 'BLASKA',
           'nom': 'Panier à linge',
           'price': '3,99',
           'ratings': '4.8'},
          {'comments': '184',
           'marque': 'BLASKA',
           'nom': 'Pelle et balayette',
           'price': '1,50',
           'ratings': '3.9'},
          {'comments': '59',
           'marque': 'TORKIS',
           'nom': 'Panier avec 30 pinces à linge',
           'price': '2,99',
           'ratings': '4.1'},
          {'comments': '117',
           'marque': 'ÅBYÅN',
           'nom': 'Éponge de toilette',
           'price': '2,99',
           'ratings': '4.2'},
          {'comments': '46',
           'marque': 'BJÄRSEN',
           'nom': 'Rideau de douche',
           'price': '1,99',
           'ratings': '4.5'},
          {'comments': '57',
           'marque': 